### 1. Базовое применение

In [291]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [292]:
data = pd.read_csv('./data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [293]:
item_features = pd.read_csv('./data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [294]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [295]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [296]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [297]:
data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [298]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

D:\Learning\LibPythonForDS\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [299]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

# ALS

In [300]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=6,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)



Wall time: 1.95 s


In [301]:
recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [302]:
[id_to_itemid[rec[0]] for rec in recs]

[1106523, 1133018, 999999, 1082185, 5569230]

In [303]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [304]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 20.7 s


0.15881466599698432

In [305]:
result.head(2)

,user_id,actual,als
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1051211, 878996, 1033142, 995242, 1082185]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1106523, 908531, 822407, 1133018, 938700]"


### Embeddings

In [306]:
model.item_factors.shape

(5001, 64)

In [307]:
model.user_factors.shape

(2500, 64)

In [308]:
# model.rank_items()

Можно очень быстро посчитать предсказания, перемножив эти 2 матрицы

In [309]:
fast_recs = model.user_factors @ model.item_factors.T 
fast_recs.shape

(2500, 5001)

In [310]:
fast_recs[0,:]

array([ 0.00451981,  0.00983432,  0.00067963, ...,  0.05714028,
       -0.0494174 ,  0.00029483], dtype=float32)

In [311]:
%%time
recommendations = model.recommend_all(N=5, 
                                      user_items=csr_matrix(user_item_matrix).tocsr(),
                                      filter_already_liked_items=True, 
                                      filter_items=None, 
                                      recalculate_user=True,
                                      show_progress=True,
                                      batch_size=500)
recommendations


Wall time: 16.7 s


array([[3019,  822, 2685,  659, 3605],
       [2297, 2747, 2134, 4337,  214],
       [1908, 2747,  557,  527, 2297],
       ...,
       [4337, 2297, 2134,  557, 3575],
       [2747, 2297, 2838,  298,  655],
       [ 557, 3679, 1317, 2447, 4054]], dtype=int32)

In [312]:
recommendations.shape

(2500, 5)

In [313]:
model.item_factors.shape

(5001, 64)

*Посмотрите также / Похожие товары*

In [314]:
example_item_row_id = 3606

In [315]:
id_to_itemid[example_item_row_id]

1101010

In [316]:
closest_items = [ id_to_itemid[row_id] for row_id, score in model.similar_items(example_item_row_id, N=5)]

In [317]:
closest_items

[1101010, 819978, 1040807, 933835, 1018740]

In [318]:
item_features[item_features.item_id.isin(closest_items)]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
5915,819978,3544,DELI,National,CHEESES,CHEESE: PROCESSED BULK,
18923,933835,3705,DELI,National,DELI MEATS,MEAT: SAUS DRY BULK,
28449,1018740,3516,DELI,National,DELI MEATS,MEAT:HAM BULK,
30927,1040807,3544,DELI,National,CHEESES,CHEESE: PROCESSED BULK,
37665,1101010,3862,DELI,National,DELI MEATS,MEAT: LUNCHMEAT BULK,


*Вашим друзьям нравится / Похожим пользователям нравится / ...*

Пользователь --> похожих пользовтелей --> рекомендовать те товары, которые купили похожие юзеры

In [319]:
model.similar_users(userid_to_id[10], N=5)

[(9, 1.0),
 (790, 0.979288),
 (354, 0.97446334),
 (1681, 0.97270465),
 (1655, 0.97170764)]

### 2. TF-IDF взвешивание

In [320]:
user_item_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [321]:
%%time

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=10)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)



Wall time: 2.09 s


In [322]:
result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

0.1575087895529863

In [323]:
(1 - 0.15/0.16) * 100

6.25

### 3. BM25 взвешивание

In [324]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

D:\Learning\LibPythonForDS\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [325]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [326]:
%%time

# model = AlternatingLeastSquares(factors=128, 
#                                 regularization=0.05,
#                                 iterations=15, 
#                                 calculate_training_loss=True, 
#                                 num_threads=4) # K - кол-во билжайших соседей

model = AlternatingLeastSquares(factors=64, 
                                regularization=0.05,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=10)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))


Wall time: 24.3 s


In [327]:
result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

0.17488699146157458

In [328]:
result.to_csv('./predictions/predictions_mf.csv', index=False)  # mf - matrix factorization

## 4. Важно искать оптимальные параметры

- regularization, iterations(+-)
- factors
- Вес (элемент в матрице user-item)

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [329]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [330]:
data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))
data_train['price'].max()

<ipython-input-330-73e31537f690>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))


499.99

In [331]:
# < 1$
data_train['price'].quantile(0.20)

0.99

In [332]:
# > 100$
data_train['price'].quantile(0.99995)

84.8129592499882

In [333]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товарыs
    
    # ...
    
def postfilter_items(user_id, recommednations):
    pass

Все эти функции отправим затем в *utils.py*

# ДЗ

1) Попытаться ответить на вопросы/выдвинуть гипотезы

**Реальность:**
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
    <br> в каждом e-mail-е есть ссылка для перехода на страницу сос счетчиком. Посчитаем % перешедших.
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?<br>
    push-уведомления использовать дешевле, но не все пользователи могут его получать. Поэтому можно сначала использовать только
    их, а на втором этапе добавить смс. Посчитать разницу, и экономическую эффективность. По поводу чеков сомневаюсь в 
    целесообразности. Если и использовать, то об этом надо как-то объявить, т.к. многие их просто не читают. К тому же, чек
    покупатель видит после покупки, а к следующему посещению может просто забыть о сообщении в нем.  
    
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?) <br>
    первым блоком 3 акционных товара (вне зависимости от того покупал раньше или нет), т.к. может купить не себе.
    вторым блоком возможно будет лучше взять топ-3 из продаваемых, но которые этот конкрентый покупатель не покупал
    третьим блоком + топ-1 одной из допустим трех категории покупаемых им товаров, но ранее не купленых. 
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?<br>
    акционные товары должны послужить стимулом начать покупки, остальные из выборки вызвать интерес.
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки<br>
    тут все сложно, поскольку некоторые скидочные программы можно проводить за счет поставщика (производителя), расходы на
    коммуникацию, за исключением смс постоянны, остается еще оплата сотрудника DS ))
    
- Cколько мы хотим зарабатывать с одного привлеченного юзера?<br>
    Здесь проще считать в процентах прироста. 
---
- А точно нужно сортировать по вероятности?<br>
    сортировать лучше внутри групп (кластеров) товаров. Например общая выборка товаров, выборка по категории, выборка по
    похожим покупателям
- Какую метрику использовать?<br>
  наверное не одну,  precision точно.
- Сколько раз в неделю отпрпавляем рассылку?<br>
    не чаще пары раз в неделю, т.к. необходимо дать время на принятие решений и не перейти в разряд спама.
- В какое время отправляем рассылку?<br>
    перед обеденным перерывом, допустим с 11-45 до 12-45, чтобы было время пррочитать и принять решение.
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?<br>
    изменять перечень акицонных товаров, смещать временной интервал для train-а, предлагать топы из разных категорий товаров 
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?<br>
    брать товары из покупаемых категорий, но более высокого ценового сегмента, предложив на них скидку.  

## Практика:

4) Поэкспериментировать с ALS (grid-search) ********
   * Подобрать лучшие параметры
   * Написать отчет об увиденном

In [334]:
model = AlternatingLeastSquares(factors=120, 
                                regularization=0.09,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=6,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['als_bm25_120_30_009'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

In [335]:
result.head()

,user_id,actual,als,als_tfidf,als_bm25,als_bm25_120_30_009
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1051211, 878996, 1033142, 995242, 1082185]","[1100972, 999999, 1082185, 1105488, 901062]","[999999, 1082185, 995242, 1100972, 1043301]","[999999, 1082185, 995242, 965766, 1100972]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1106523, 908531, 822407, 1133018, 938700]","[1106523, 999999, 1133018, 951590, 1029743]","[999999, 1092026, 951590, 883404, 5569230]","[1053690, 999999, 1133018, 951590, 840601]"
2,5,"[913077, 1118028, 1386668]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 6534178, 1029743, 995242]","[999999, 1082185, 860248, 981760, 1404121]","[999999, 1082185, 995242, 981760, 1058997]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[1007195, 878996, 1051516, 1024306, 1023720]","[878996, 1024306, 1051516, 1007195, 860776]","[1082185, 1024306, 878996, 857006, 999999]","[1082185, 1023720, 1024306, 866211, 878996]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[999999, 1082185, 849843, 1126899, 1058997]","[999999, 1082185, 6534178, 826249, 1029743]","[999999, 1082185, 849843, 1130111, 6944571]","[999999, 1082185, 6944571, 828867, 1122358]"


In [336]:
result.apply(lambda row: precision_at_k(row['als_bm25_120_30_009'], row['actual']), axis=1).mean()

0.19326971371169982

factors=120 iterations=15 regularization=0.09 - 0.19326971371169982 

матрицы большей чем 120 разрядности не дают заметного прироста качества, видимо ввиду того, что точность уравнений достигает достаточного уровня. Выбор окэффициента регулиризации обусловлен максимальным значеним, при котором качество не падает на уровне 4 знака. Максимум выбрал чтобы максимально штрафовать нетипичные случаи (например парсер-ботов и охотников за скидками)